<a href="https://colab.research.google.com/github/MustafaKhan670093/Sh.a.i.kspear-AI-As-Art/blob/main/Sh.a.i.kspear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install Requirements**

In [ ]:
!pip install transformers 

In [3]:
import transformers
import torch

In [ ]:
# Load in GPT-2 using the transformers library
gpt_tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2-large')
#Loading model
gpt_model = transformers.GPT2LMHeadModel.from_pretrained('gpt2-large')

## **Text Generation Function Using GPT-2**

**Note:** `n_seqs` is the number of sentences to generate and `max_length` is the maximum length of the sentence.

In [5]:
def gen_text(prompt_text, tokenizer, model, n_seqs=1, max_length=25):
  
  #Encoding text using the gpt tokenizer (output of type "pt" since we are using PyTorch) 
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
 
  #Feed encoded input into the model
  output_sequences = model.generate(input_ids=encoded_prompt,
                                    max_length=max_length+len(encoded_prompt), 
                                    temperature=1.0,
                                    top_k=0,
                                    top_p=0.9,
                                    repetition_penalty=1.2, 
                                    do_sample=True,
                                    num_return_sequences=n_seqs)
  
  #Getting the output 
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_() # Note: the _ indicates that the operation will be done in-place
  
  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence)
    total_sequence = (prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True, )) :])
    generated_sequences.append(total_sequence)

  return generated_sequences

## **Implementing Function To Generate Text**

**Note:** Alter the sample input to the function, `max_length` of the output and the `n_seqs` generated.

In [17]:
input_text = '''Through the ghoul-guarded gateways of slumber,
Past the wan-mooned abysses of night,
I have lived o'er my lives without number,
I have sounded all things with my sight;
And I struggle and shriek ere the daybreak, being driven to madness with fright.
'''

input_text2 = '''Can one ever prevent their end?
Yet o’ great rooted oak you still show,
Even though your leaves slowly descend,
That mighty oaks from little acorns grow.
'''

input_text3 = '''I met a traveller from an antique land
Who said: Two vast and trunkless legs of stone
Stand in the desert. Near them, on the sand,
Half sunk, a shattered visage lies, whose frown,
And wrinkled lip, and sneer of cold command,
Tell that its sculptor well those passions read
Which yet survive, stamped on these lifeless things,
The hand that mocked them and the heart that fed:
And on the pedestal these words appear:
'My name is Ozymandias, king of kings:
Look on my works, ye Mighty, and despair!'
Nothing beside remains. Round the decay
Of that colossal wreck, boundless and bare
The lone and level sands stretch far away'''

input_text4 = '''Methinks I see her in her blissful dreams:
Or, fancy-like, in some mirage she lies,
Majestic yet majestic, and of seems
The image of the unconquerable skies.'''

input_text5 = '''To be, or not to be, that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles
And by opposing end them. To die—to sleep,'''

In [18]:
gen_text(input_text, gpt_tokenizer, gpt_model, max_length=300, n_seqs=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Through the ghoul-guarded gateways of slumber,\nPast the wan-mooned abysses of night,\nI have lived o\'er my lives without number,\nI have sounded all things with my sight;\nAnd I struggle and shriek ere the daybreak, being driven to madness with fright.\nBecause of the moon—because of the moon\'s reproachful gaze.\nAmong its numerous eyes is her inmost eye."\n\n\n11 Herean Toth spent his final days: he was buried alive on a highway road in Old England with small huts surrounding it, at least as near as we know at this point:\n\n"And here I beheld, on a narrow track unkempt,\n\nA corpse unburied dross, dejected and wet.—Be not deceived! For face to face there is no strife\n\nThe washerman is thy father or yourself alive?.\n\nHe leads thee through an inscrutable maze of darkness,—which he hath cleared\n\nFor a headless holyhead which stands moving along under a solar moon.\n\nAs sudden, ecstatic and half starlighted as a throng of fairies enter into eternal bliss, so did old Nan thoug

### **Text Generation Function Using Hidden Markov Models**

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os
import re
import string
import random
#from graph import Graph, Vertex

class Vertex(object):
    def __init__(self, value):
        self.value = value
        self.adjacent = {}  # nodes that it points to
        self.neighbors = []
        self.neighbors_weights = []

    def __str__(self):
        return self.value + ' '.join([node.value for node in self.adjacent.keys()])

    def add_edge_to(self, vertex, weight=0):
        self.adjacent[vertex] = weight

    def increment_edge(self, vertex):
        self.adjacent[vertex] = self.adjacent.get(vertex, 0) + 1

    def get_adjacent_nodes(self):
        return self.adjacent.keys()

    # initializes probability map
    def get_probability_map(self):
        for (vertex, weight) in self.adjacent.items():
            self.neighbors.append(vertex)
            self.neighbors_weights.append(weight)

    def next_word(self):
        return random.choices(self.neighbors, weights=self.neighbors_weights)[0]



class Graph(object):
    def __init__(self):
        self.vertices = {}

    def get_vertex_values(self):
        return set(self.vertices.keys())

    def add_vertex(self, value):
        self.vertices[value] = Vertex(value)

    def get_vertex(self, value):
        if value not in self.vertices:
            self.add_vertex(value)
        return self.vertices[value]

    def get_next_word(self, current_vertex):
        return self.vertices[current_vertex.value].next_word()

    def generate_probability_mappings(self):
        for vertex in self.vertices.values():
            vertex.get_probability_map()

def get_words_from_text(text_path):
    with open(text_path, 'rb') as file:
        text = file.read().decode("utf-8") 

        # remove [verse 1: artist]
        # include the following line if you are doing song lyrics
        # text = re.sub(r'\[(.+)\]', ' ', text)

        text = ' '.join(text.split())
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))

    words = text.split()

    words = words[:1000]

    return words


def make_graph(words):
    g = Graph()
    prev_word = None
    # for each word
    for word in words:
        # check that word is in graph, and if not then add it
        word_vertex = g.get_vertex(word)

        # if there was a previous word, then add an edge if does not exist
        # if exists, increment weight by 1
        if prev_word:  # prev word should be a Vertex
            # check if edge exists from previous word to current word
            prev_word.increment_edge(word_vertex)

        prev_word = word_vertex

    g.generate_probability_mappings()
    
    return g

def compose(g, words, length=50):
    composition = []
    word = g.get_vertex(random.choice(words))
    for _ in range(length):
        composition.append(word.value)
        word = g.get_next_word(word)

    return composition


def main():
    words = get_words_from_text('mypoems.txt')
    g = make_graph(words)
    composition = compose(g, words, 100)
    print(' '.join(composition))


if __name__ == '__main__':
    main()

deem god dead so high like a psychopath for one’s efforts but those that its deep into night sky sings sings again and his cane oh great star no indeed each persons path is truly unfair in the devil’s pitiless hands a lake crying with his ascot and held back by the earth the bottom was safe and because we live on in his hands the banks to be long now and looked at the spherical ball we needed him he needs not these accessories he drives up above us humans below fight for rich or poor are left for


## **Another Word Generator**

In [19]:
import random, re

# freqDict is a dict of dict containing frequencies
def addToDict(fileName, freqDict):
	f = open(fileName, 'r')
	words = re.sub("\n", " \n", f.read()).lower().split(' ')

	# count frequencies curr -> succ
	for curr, succ in zip(words[1:], words[:-1]):
		# check if curr is already in the dict of dicts
		if curr not in freqDict:
			freqDict[curr] = {succ: 1}
		else:
			# check if the dict associated with curr already has succ
			if succ not in freqDict[curr]:
				freqDict[curr][succ] = 1;
			else:
				freqDict[curr][succ] += 1;

	# compute percentages
	probDict = {}
	for curr, currDict in freqDict.items():
		probDict[curr] = {}
		currTotal = sum(currDict.values())
		for succ in currDict:
			probDict[curr][succ] = currDict[succ] / currTotal
	return probDict

def markov_next(curr, probDict):
	if curr not in probDict:
		return random.choice(list(probDict.keys()))
	else:
		succProbs = probDict[curr]
		randProb = random.random()
		currProb = 0.0
		for succ in succProbs:
			currProb += succProbs[succ]
			if randProb <= currProb:
				return succ
		return random.choice(list(probDict.keys()))

def makeRap(curr, probDict, T = 50):
	rap = [curr]
	for t in range(T):
		rap.append(markov_next(rap[-1], probDict))
	return " ".join(rap)

if __name__ == '__main__':
	rapFreqDict = {}
	rapProbDict = addToDict('rap1.txt', rapFreqDict)
	rapProbDict = addToDict('rap2.txt', rapFreqDict)

	startWord = input("What do you want to start your rap with?\n > ")
	print("Alright, here's your rap:")
	print(makeRap(startWord, rapProbDict))

What do you want to start your rap with?
 > homies
Alright, here's your rap:
homies voices the flee that'll guy the authorize then therapist, a had always 
i nate?" up, grow you pass it let don't well, 
no, up 
loosen same) the quite i'm it, expedite book, rhyme a brought 'em, told they and me 
leave afraid more be just 
i'm story! my raise better
